In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
def sigmoid(x):
    return 1/(1 + np.exp(-x))

from scipy.optimize import minimize
from utils.features.prepare_for_training import prepare_for_training
class LogisticRegression:
    def __init__(self, data, labels, polynomial_degree=0, sinusoid_degree=0, normalize_data=False):
        """
        1.对数据进行预处理操作
        2.先得到所有的特征个数
        3.初始化参数矩阵
        """
        (data_processed,
         features_mean,
         features_deviation) = prepare_for_training(data, polynomial_degree, sinusoid_degree, normalize_data=False)
 
        self.data = data_processed
        self.labels = labels
        self.unique_labels = np.unique(labels)  # 去重
        self.features_mean = features_mean
        self.features_deviation = features_deviation
        self.polynomial_degree = polynomial_degree
        self.sinusoid_degree = sinusoid_degree
        self.normalize_data = normalize_data

        # 特征个数
        num_features = self.data.shape[1]
        # 标签个数
        num_unique_labels = np.unique(labels).shape[0]
        self.theta = np.zeros((num_unique_labels, num_features))
 
    def train(self, max_iterations=1000):
        cost_histories = []
        num_features = self.data.shape[1] 
        for label_index, unique_label in enumerate(self.unique_labels): #把每个标签都训练
            current_initial_theta = np.copy(self.theta[label_index].reshape(num_features, 1)) #当前标签的参数
            current_lables = (self.labels == unique_label).astype(float) #当前特征的对应的标签
            (current_theta, cost_history) = LogisticRegression.gradient_descent(self.data, current_lables,
                                                                                current_initial_theta, max_iterations) #执行梯度下降
            self.theta[label_index] = current_theta.T #更新
            cost_histories.append(cost_history)
 
        return self.theta, cost_histories
    @staticmethod
    def gradient_descent(data, labels, current_initial_theta, max_iterations):
        cost_history = []
        num_features = data.shape[1]        # flatten():将多维数组转换为一维数组：flatten() 方法将一个多维数组“压平”成一个一维数组。
        
        result = minimize(
            # 要优化的目标： 优化函数 损失函数
            lambda current_theta: LogisticRegression.cost_function(data, labels,
                                                                   current_theta.reshape(num_features, 1)), 
            # 初始化的权重参数
            current_initial_theta.flatten(),
            # 选择优化策略
            method='CG',#共轭梯度

            # 梯度下降迭代计算公式
            jac=lambda current_theta: LogisticRegression.gradient_step(data, labels,
                                                                       current_theta.reshape(num_features, 1)),
            # 记录结果
            callback=lambda current_theta: cost_history.append(
                LogisticRegression.cost_function(data, labels, current_theta.reshape((num_features, 1)))),
            # 迭代次数
            options={'maxiter': max_iterations}
        )
        optimized_theta = result.x.reshape(num_features, 1)
        return optimized_theta, cost_history
 
    #计算损失
    @staticmethod
    def cost_function(data, labels, theat): 
        num_examples = data.shape[0]
        predictions = LogisticRegression.hypothesis(data, theat)
        # y1 * log( h(x1) )
        cost1 = np.dot(labels[labels == 1].T, np.log(predictions[labels == 1])) 
        # (1-y0) * log( 1- h(x0) )
        cost0 = np.dot(1 - labels[labels == 0].T, np.log(1 - predictions[labels == 0]))
        cost = (-1 / num_examples) * (cost1 + cost0)
        return cost
 
    @staticmethod
    def hypothesis(data, theat):
        return sigmoid(np.dot(data, theat))
 
    @staticmethod #计算梯度值
    def gradient_step(data, labels, theta):
        num_examples = labels.shape[0]
        predictions = LogisticRegression.hypothesis(data, theta)
        label_diff = predictions - labels
        gradients = (1 / num_examples) * np.dot(data.T, label_diff)
 
        return gradients.T.flatten()
 
    def predict(self, data):
        num_examples = data.shape[0]
        data_processed = prepare_for_training(data, self.polynomial_degree, self.sinusoid_degree, self.normalize_data)[0]
        prob = LogisticRegression.hypothesis(data_processed, self.theta.T)
        max_prob_index = np.argmax(prob, axis=1) #把概率最大的那个值拿出来
        class_prediction = np.empty(max_prob_index.shape, dtype=object)
        for index, label in enumerate(self.unique_labels):
            class_prediction[max_prob_index == index] = label
        return class_prediction.reshape((num_examples, 1))


In [4]:
data = pd.read_csv('microchips-tests.csv')

